In [ ]:
from diffusers import AutoencoderKL

In [ ]:
akl = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae").to('cuda')

In [ ]:
from PIL import Image
from torchvision import transforms

img = Image.open("./celeb/29992.jpg")
img = transforms.ToTensor()(img)
print(img.shape)

In [ ]:
import matplotlib.pyplot as plt

# 이미지 표시
plt.imshow(img.permute(1,2,0).numpy())
plt.axis('off')  # 축 제거
plt.show()

In [ ]:
res = akl.encode(img.unsqueeze(dim=0).to('cuda'))

In [ ]:
print(res['latent_dist'].mean.shape)

In [ ]:
decoded = akl.decode(res['latent_dist'].mean)

In [ ]:
decoded[0].shape

In [ ]:
array = decoded[0][0].permute(1,2,0).cpu().detach().numpy()
array.shape

In [ ]:
import matplotlib.pyplot as plt

# 이미지 표시
plt.imshow(array)
plt.axis('off')  # 축 제거
plt.show()

In [ ]:
len(res)

In [ ]:
import torch
from ldm.autoencoder import AutoencoderKL

In [ ]:
ddconfig = {
    "double_z":True,
    "z_channels":16,
    "resolution":256,
    "in_channels":3,
    "out_ch":3,
    "ch":128,
    "ch_mult":[1,1,2,2,4],
    "num_res_blocks":2,
    "attn_resolutions":[16],
    "dropout":0.0,
}

lossconfig = {
    "target":".losses.LPIPSWithDiscriminator",
    "params":{
        "disc_start":50001,
        "kl_weight":1.0e-06,
        "disc_weight":0.5
    },
}

In [ ]:
akl = AutoencoderKL(embed_dim=16, ddconfig=ddconfig, lossconfig=lossconfig)

In [ ]:
from PIL import Image
from torchvision import transforms

img = Image.open("./celeb/29992.jpg")
img = transforms.ToTensor()(img)

In [ ]:
img.shape

In [ ]:
latent = akl.encoder(img.unsqueeze(dim=0))
print(latent.shape)

In [ ]:
res = akl(img.unsqueeze(dim=0))

In [ ]:
array = res[0][0].permute(1,2,0).detach().numpy()

In [ ]:
from model.unet import Unet


unet = Unet(dim=128, mults=[1, 2, 2, 4], channel_scale=4).to(device)

In [ ]:
def linear_beta_schedule(timesteps=1000):
    betas = torch.linspace(beta_0, beta_T, steps=timesteps+1)

    alphas = 1. - betas
    alphas_bar = torch.cumprod(alphas, dim=0)

    return betas, alphas, alphas_bar

In [ ]:
batch_size = 128
learning_rate = 0.00005
epochs = 1000
total_timesteps = 1000
beta_0 = 0.0001
beta_T = 0.02

betas, alphas, alphas_bar = linear_beta_schedule(total_timesteps)

def x_t_sample(x_0, timesteps, noise):
    return torch.stack([torch.sqrt(alphas_bar[t])*x_0[idx] + torch.sqrt(1-alphas_bar[t])*noise[idx] for idx, t in enumerate(timesteps)])

def x_t_1_sample(x_t, timesteps, predicted_noise, z):
    moved_mean = torch.stack([x_t[idx] - (1-alphas[t])/(torch.sqrt(1-alphas_bar[t])) * predicted_noise[idx] for idx, t in enumerate(timesteps)])
    return torch.stack([1/torch.sqrt(alphas[t]) * moved_mean[idx] + torch.sqrt(betas[t]) * z[idx] for idx, t in enumerate(timesteps)])

In [ ]:
print(torch.max(decoded[0][0]))
print(torch.min(decoded[0][0]))

In [ ]:
import matplotlib.pyplot as plt

x_t=x_0
for i in range(1000):
    time = torch.tensor(i, device=device).unsqueeze(dim=0)
    added_noise = torch.randn_like(x_0)
    x_t = x_t_sample(x_t, time, added_noise)

    if i%100==99:
        decoded = akl.decode(x_t)
        array = decoded[0][0].permute(1,2,0).cpu().detach().numpy()
        # 이미지 표시
        plt.imshow(array)
        plt.axis('off')  # 축 제거
        plt.show()

In [ ]:
array = (decoded[0][0]/2+0.5).permute(1,2,0).cpu().detach().numpy()
# 이미지 표시
plt.imshow(array)
plt.axis('off')  # 축 제거
plt.show()